# Problèmes et Extensions

## Problèmes
- javascript runs fail
- faire de nouvelles sorties pour les kalabas sans séparateurs de mots
 - radicaux seuls
 - découpage en syntagmes pour l'exercice sur l'ordre des syntagmes

## Extensions
- traitement du paucal
 - 2,3,4,5 ou suffixe P sur le nom sans numéral
- ajout des compléments/ajouts multiples
 - séparation par ;
 - gestion des dislocations gauches dans les multiples

### Javascript from :
https://stackoverflow.com/questions/12544056/how-to-i-get-the-current-ipython-notebook-name

### Imports

In [73]:
from os.path import expanduser
#from cellbell import ding
from itertools import groupby
import codecs, optparse
import re, random
import sys,os,time
import string
import yaml, warnings
import ParFuMor as PFM
from ParFuMor import *
import pickle,copy


In [74]:
%%javascript
var kernel = IPython.notebook.kernel;
var thename = window.document.getElementById("notebook_name").innerHTML;
var command = "theNotebook = " + "'"+thename+"'";
kernel.execute(command);

<IPython.core.display.Javascript object>

In [75]:
try:
    print(theNotebook)
except:
    pass

### Gestion partagée des numéros à traiter & gestion des variantes

Le block suivant permet de partager les numéros à traiter entre les différents Notebooks.
- %store -r variable lit la variable dans le stock
- %store variable stocke la variable

Après la réussite de la variante de base, on change automatiquement la variante à -Corr pour générer les fichiers de solutions.

In [76]:
# numerosKalaba=[5]
# %store numerosKalaba
%store -r numerosKalaba typeKalaba
variante=""
if 'nextVariante' in globals():
    variante=nextVariante
print variante
print numerosKalaba, typeKalaba

-Corr
[18] Kanonik


In [77]:
home = expanduser("~")

if typeKalaba!="Kanonik":
    numeroKalaba="22-K%d/"%numerosKalaba[0]
    repertoire=home+"/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/00-ProjetKalaba/"
    serie=repertoire+numeroKalaba
else:
    repertoire=home+"/Library/Mobile Documents/com~apple~CloudDocs/Downloads/"
    serie=repertoire+"23-Kanoniks/"
    serie=serie+"Kanonik-%02d-ok/"%numerosKalaba[0]
print serie

/Users/gilles/Library/Mobile Documents/com~apple~CloudDocs/Downloads/23-Kanoniks/Kanonik-18-ok/


In [78]:
# -*- coding: utf8 -*-
def ding():
    os.system('afplay /System/Library/Sounds/Submarine.aiff')
    
    
complementPhrases=""


### Kalaba 3 et 5
Pour le kalaba 3 et le 5, pas de séparateur entre les mots
Pour le kalaba 5, mettre Mahira dans Mots.csv et Agathos dans Phrases.csv (pour la gestion du puzzle)

In [79]:
#########################VARIABLES##########################################
version=os.path.basename("__file__")
time_stamp='%s' % time.strftime("%y%m%d-%H%M")
debug=0
debug_now=0

#casNombreDet=True
RightLeft=True

if (numerosKalaba[0] in [3,5]) and variante=="" and typeKalaba!="Kanonik":
    separateurPhonoCloze=""
else:
    separateurPhonoCloze=" "
# separateurPhonoCloze=" "    
separateurMots=separateurPhonoCloze
print "séparateur",separateurMots
#separateurMots=" "
marqueurCommentaire="%"
print_no=True
print_taches=False
print_coffee=False
print_commands=True
print_ortho=True
print_phono=True
print_glose=False        #False
print_radicaux=False     #False
#if separateurMots=="":
#    print_glose=False
if variante=="-Corr":
    print_taches=False
    print_coffee=False
    print_commands=True
    print_ortho=True
    print_phono=True
    print_glose=True    
if print_glose:
    separateurMots=" "
if print_glose+print_ortho+print_phono>1:
    print_phrases=True
else:
    print_phrases=False
print_lexique=True
print_cloze=True
print_racines=True

séparateur  


In [80]:
print_glose+print_ortho+print_phono

3

In [81]:
prononciationBegin=[
    "\\begin{center}",
        "\\begin{tabular}{lcc}",
        "\\toprule",
        u"Graphie & Prononciation & Mot français \\\\",
        "\\midrule"
        ]
prononciationEnd=[
        "\\bottomrule",
        "\\end{tabular}",
    "\\end{center}"
]

In [82]:
with open(serie+"Gloses.yaml", 'r') as stream:
    gloses=yaml.safe_load(stream)
with open(serie+"Stems.yaml", 'r') as stream:
    stems=yaml.safe_load(stream)
with open(serie+"Phonology.yaml", 'r') as stream:
    phonology=yaml.safe_load(stream)
with open(serie+"MorphoSyntax.yaml", 'r') as stream:
    morphosyntax=yaml.safe_load(stream)
with open(serie+"Clozes.txt", 'r') as stream:
    clozeLines=stream.readlines()

discrimineur=[]
discriminant={}
for line in clozeLines:
    line=line.strip()
    if not line.startswith("#"):
        elementsCloze=line.split(";")
        discriminant[elementsCloze[0]]=";".join([element for element in elementsCloze[1:] if element!=elementsCloze[5]])

In [83]:
#discriminant

In [84]:
defaultCols=2
defaultLong=0
maxChunk=48
dimensionsTableaux={i:{} for i in gloses}
for categorie in gloses:
    if "Dimensions" in morphosyntax:
        if "maxChunk" in morphosyntax["Dimensions"]:
            maxChunk=morphosyntax["Dimensions"]["maxChunk"]
        print maxChunk
        if categorie in morphosyntax["Dimensions"] and morphosyntax["Dimensions"][categorie]:
#            print categorie
            if "cols" in morphosyntax["Dimensions"][categorie]:
                dimensionsTableaux[categorie]["cols"]=morphosyntax["Dimensions"][categorie]["cols"]
            else:
                dimensionsTableaux[categorie]["cols"]=defaultCols
            if "long" in morphosyntax["Dimensions"][categorie]:
                dimensionsTableaux[categorie]["long"]=morphosyntax["Dimensions"][categorie]["long"]
            else:
                dimensionsTableaux[categorie]["long"]=defaultLong
        else:
            dimensionsTableaux[categorie]["cols"]=defaultCols
            dimensionsTableaux[categorie]["long"]=defaultLong
    else:
        dimensionsTableaux[categorie]["cols"]=defaultCols
        dimensionsTableaux[categorie]["long"]=defaultLong
if print_radicaux:
    nomTableaux="Tableaux-Gloses.yaml"
else:
    nomTableaux="Tableaux.yaml"    
with open(serie+nomTableaux, 'r') as stream:
    tableaux=yaml.safe_load(stream)
with open(serie+"Hierarchie-S2.pkl", 'rb') as input:
   PFM.hierarchieCF = pickle.load(input)
with open(serie+"Lexique-S2.pkl", 'rb') as input:
   PFM.lexique = pickle.load(input)
with open(serie+"Regles-S2.pkl", 'rb') as input:
   PFM.regles = pickle.load(input)

28
28
28
28
28
28


In [85]:
[morphosyntax["Contractions"][x] for x in morphosyntax["Contractions"]]

[[u'\xe0', 'les'], ['de', 'le'], ['de', 'les'], [u'\xe0', 'le'], ['de']]

#### Définition des entêtes

In [86]:
#########################CONSTANTS##########################################
head = [
"\\begin{tabular}[t]{|l|l|l|}",
"\\addlinespace[-1.0em]\\hline",
"Mot & Roman & Glose  \\\\",
"\\hline\\strutgh{14pt}%"
]
head_n = [
"\\begin{tabular}[t]{|l|c|c|c|}",
"\\addlinespace[-1.0em]\\hline",
"Nom & Genre & C\\indice{1}C\\indice{2}C\\indice{3} & V\\indice{L}  \\\\",
"\\hline\\strutgh{14pt}%"
]
head_v = [
"\\begin{tabular}[t]{|l|c|c|}",
"\\addlinespace[-1.0em]\\hline",
"Verbe & Type & C\\indice{1}C\\indice{2}C\\indice{3} \\\\",
"\\hline\\strutgh{14pt}%"
]

In [87]:
tail = [
"\\hline"
"\\end{tabular}\\columnbreak\\vfill"
]

#### Définition des structures pour impression

In [88]:
def accumulerMots(mot):
    accumulateur.append(mot)
    return

def ajouterExemple(exemple,printBool=False):
    if printBool:
        print exemple
    exemples.append(exemple.strip())
    del accumulateur[:]
    return
def ajouterVocabulaire(terme,printBool=False):
    if printBool:
        print terme
    vocabulaire.append(terme.strip())
    return

#### Définition des segments

In [89]:
consonnes=phonology["consonnes"]
voyelles=phonology["voyelles"]
gabarits=phonology["gabarits"]
derives=phonology["derives"]
nom_classe=phonology["nom_classe"]
nom_apo=phonology["apophonies"]

nom_mut=phonology["mutations"]

#### Définition des catégories

In [90]:
sujetVerbe=[]
if "Cas" in gloses["NOM"]:
    if "Nom" in gloses["NOM"]["Cas"]:
        sujetVerbe.append("Nom")
    if "Erg" in gloses["NOM"]["Cas"]:
        sujetVerbe.append("Abs")

In [91]:
#sujetVerbe

In [92]:
attributsMots=set()
valAttributs={}
for categorie in gloses:
    if gloses[categorie]:
        for element in gloses[categorie]:
            attributsMots.add(element)
            if gloses[categorie][element]:
                if not element in valAttributs:
                    valAttributs[element]=[]
                for attribut in gloses[categorie][element]:
                    if not attribut in valAttributs[element]:
                        valAttributs[element].append(attribut)

In [93]:
attributsMots

{'Genre', 'Nombre', 'Pers', 'Temps'}

In [94]:
boolAttribut={}
flexCategories=[element for element in gloses if gloses[element]]
for element in flexCategories:
    boolAttribut[element]={key:key in gloses[element] for key in attributsMots}
flexCategories

['NOM', 'VER', 'ADJ', 'PRO', 'DET']

In [95]:
boolAttribut

{'ADJ': {'Genre': True, 'Nombre': True, 'Pers': False, 'Temps': False},
 'DET': {'Genre': True, 'Nombre': True, 'Pers': False, 'Temps': False},
 'NOM': {'Genre': True, 'Nombre': True, 'Pers': False, 'Temps': False},
 'PRO': {'Genre': True, 'Nombre': True, 'Pers': False, 'Temps': False},
 'VER': {'Genre': True, 'Nombre': False, 'Pers': True, 'Temps': True}}

attributsDetAdjNom=["Genre","Nombre","Cas"]
valAttribut={}
for attribut in attributsDetAdjNom:
    if attribut in gloses["N"]:
        valAttribut[attribut]=gloses["N"][attribut]
    elif attribut in gloses["ADJ"]:
        valAttribut[attribut]=gloses["ADJ"][attribut]
    elif attribut in gloses["DET"]:
        valAttribut[attribut]=gloses["DET"][attribut]
    else:
        valAttribut[attribut]=[]
boolAttribut={}
for element in ["DET","ADJ","N"]:
    boolAttribut[element]={key:key in gloses[element] for key in attributsDetAdjNom}

In [96]:
#types=gloses["V"]["CF"]
if "Cas" in morphosyntax:
    casSyntagmes=morphosyntax["Cas"]
else:
    casSyntagmes=""
lexiquePrepositions=[stems["PREP"][x][0] for x in stems["PREP"]]
casPreposition={}
for preposition in lexiquePrepositions:
    prep=preposition.upper()
    if casSyntagmes and prep in casSyntagmes:
        casPreposition[preposition]=casSyntagmes[prep].capitalize()
    elif casSyntagmes and "PREP" in casSyntagmes:
        casPreposition[preposition]=casSyntagmes["PREP"].capitalize()
    else:
        casPreposition[preposition]=""
        
i=2
verbe_forme={}
for forme in morphosyntax["VER"]["FormesBase"]:
    verbe_forme[i]=forme
    i+=1

In [97]:
#valAttribut

#### Remplacement des numéros de personnes pour les noms de macro LaTeX

In [98]:
def remplacerPersonnes(chaine):
    chaine.replace("1","Un")
    return chaine

In [99]:
remplacerPersonnes("1")

'1'

In [100]:
def recoder(chaine,table):
    if type(chaine)==str:
        temp=unicode(chaine.decode('utf8')).translate(table)
        result=temp.encode('utf8')
    elif type(chaine)==unicode:
        result=chaine.translate(table)
    return result

accentedIn = unicode(phonology["translations"]["deaccent"]["in"])
deaccentIn = [ord(char) for char in accentedIn]
deaccentOut = unicode(phonology["translations"]["deaccent"]["out"])
deaccent = dict(zip(deaccentIn, deaccentOut))

deligatures=phonology["translations"]["deligatures"]

In [101]:
syntagmes=morphosyntax["Syntagmes"]
syntagmesLR=copy.deepcopy(syntagmes)
print syntagmesLR
syntagmesRL=copy.deepcopy(syntagmes)
for element in syntagmesRL:
    syntagmesRL[element].reverse()
nomFonctions=syntagmes["Phrase"]

{'Phrase': ['COMP', 'VER', 'AJOUT', 'SUJ', 'OBJ', 'IND'], 'GP': ['PREP', 'GN'], 'GADJ': ['ADV', 'ADJ', 'GP'], 'GN': ['GP', 'NOM', 'DET', 'GADJ']}


In [102]:
"LR",syntagmesLR, "RL",syntagmesRL

('LR',
 {'GADJ': ['ADV', 'ADJ', 'GP'],
  'GN': ['GP', 'NOM', 'DET', 'GADJ'],
  'GP': ['PREP', 'GN'],
  'Phrase': ['COMP', 'VER', 'AJOUT', 'SUJ', 'OBJ', 'IND']},
 'RL',
 {'GADJ': ['GP', 'ADJ', 'ADV'],
  'GN': ['GADJ', 'DET', 'NOM', 'GP'],
  'GP': ['GN', 'PREP'],
  'Phrase': ['IND', 'OBJ', 'SUJ', 'AJOUT', 'VER', 'COMP']})

In [103]:
contractions=morphosyntax["Contractions"]
for contraction in contractions:
    temp=[]
    for element in contractions[contraction]:
        if isinstance(element,unicode):
            temp.append(element)
        else:
            temp.append(element.decode("utf8"))
    contractions[contraction]=temp

In [104]:
syllabes=phonology["syllabes"]

In [105]:
attributsFlexVerbe=[glosesVerbe for attributFlex in gloses["VER"] for glosesVerbe in gloses["VER"][attributFlex]]
attributsFlexVerbe

['PRS', 'FUT', '3Sg', '3Du', '3Pl', 'Ani', 'Ina', 'Hum']

In [106]:
def taches():
    '''
    seuil1 pour avoir une tâche
    seuil2 pour avoir plusieurs tâches
    '''
    seuil1=8
    seuil2=14
    def makeStain():
        seed=random.randint(1,1000)
        x=random.gauss(10,5)-1
        y=random.gauss(2,1)
        minimum=random.gauss(.2,.1)+.1
        maximum=random.gauss(.1,.05)+.5
        return "\\taches{%s}{%s}{%s}{%s}{%s}"%(seed,x,y,minimum,maximum)

    if print_taches:
        n=random.gauss(10,2.5)
        if n<seuil1:
            return ""
        elif n<=seuil2:
            return makeStain()
        else:
            nTaches=int(n-seuil1)
            stains=""
            for i in range(nTaches):
                stains+=makeStain()
            return stains
    else:
        return ""

### Tableaux des formes utilisées

In [107]:
cellules={c:set() for c in "NOM VER ADJ DET PRO".split(" ")}

In [108]:
def faire_tableau(tableau,tab=(head,tail,"")):
    if len(tableau)==0: return
    comment=tab[2]
    for element in tab[0]:
        ajouterVocabulaire(comment+element)
    for element in tableau:
        ajouterVocabulaire(comment+element)
    for element in tab[1]:
        ajouterVocabulaire(comment+element)

In [109]:
def print_tableaux(cols,recuTableau,texte="",debut=0,tab=(head,tail,""),boolEchantillon=True):
    tableau=sorted(list(set(recuTableau)))
    if debug: print "recuTableau",(recuTableau)
#    print (tableau)
    ajouterVocabulaire(tab[2]+"\\begin{multicols}{"+str(cols)+"}")
#    if texte!="":
    (table,reste)=filtrer_tableau(tableau,texte)
#    print reste
#    else:
#        (table,reste)=tableau
#        reste=[]
    chunk=(len(table)-debut*cols)/cols+1
    faire_tableaux(table,debut,cols,tab)
    ajouterVocabulaire(tab[2]+"\\end{multicols}")
    echantillon=min(len(reste),3)
    if echantillon>0 and boolEchantillon:
        for element in random.sample(reste,echantillon):
#            print element
            colonnes=element.split("&")
            graphie=colonnes[0].strip()
            phonologie=colonnes[1].strip()
            glose=colonnes[2].strip().strip("\\")
            prononciationExtrait.append(graphie+"&")
            prononciationExtrait.append("\\blanc{%s}"%phonologie+"&")
            prononciationExtrait.append("\\blanc{%s}\\\\"%glose)
            if debug: print "".join(prononciationExtrait)

In [110]:
def faire_tableaux(tableau,debut=16,nombre=1,tab=(head,tail,"")):
    reste=[]
    if debug: print nombre,debut,tableau
    if debut!=0:
        for i in range(nombre):
            faire_tableau(tableau[debut*i:debut*(i+1)],tab)
        table=tableau[debut*nombre:]
    else:
        table=tableau
    longueur=len(table)
    chunk=longueur/nombre+1
    if debug: print "CHUNKING : ",longueur, nombre, chunk, table
    if chunk<maxChunk:
        chunks=chunk
    else:
        chunks=maxChunk
        reste=table[maxChunk*nombre:]
    if debug: print "RESTE : ", chunk, reste
    for i in range(nombre):
        faire_tableau(table[chunks*i:chunks*(i+1)],tab)
    if reste:
        faire_tableaux(reste,0,nombre,tab)

In [111]:
def filtrer_tableau(tableau,filtre):
    presents=[]
    absents=[]
    for line in tableau:
        elements=line.split(" ")
        cleElement=elements[0].replace("\\","")
        if elements[0] in filtre:
            if not discriminant[cleElement] in discrimineur:
                discrimineur.append(discriminant[cleElement])
    #                print "présent",elements[0]
            presents.append(line)
        else:
#                print "absent",elements[0]
            absents.append(line)
    return (presents,absents)

### Introduire les pronoms kalaba
- identifier qu'il s'agit d'un pronom
- parser le référent

In [112]:
def getNounNumber(nom,nombre):
    if nom[len(nom)-1] in ["s","x"]:
        if nombre=="": nombre="PL"
    elif nom[len(nom)-1] in ["P","D"]:
        if nombre=="": 
            if "Pau" in valAttributs["Nombre"]:
                nombre="PAU"
            elif "Du" in valAttributs["Nombre"] and nom[len(nom)-1]=="D":
                nombre="DU"
            else:
                nombre="PL"
    else:
        if nombre=="": nombre="SG"
    return nombre

def getNounGender(ref):
    nom=ref.rstrip("P").rstrip("D")
    nomLexeme=PFM.lexique.formeLexeme[nom][0]
    classesNom=nomLexeme.split('.')[1:]
    print classesNom
    proGender=[classeElement for classeElement in classesNom if classeElement in gloses["NOM"]["Genre"]][0]
    return proGender, classesNom


def getRef(pronom):
    m=re.match("^(.*)#(.*)#",pronom)
    if m:
        pro=m.group(1)
        ref=m.group(2)
    else:
        print "mauvaise notation pour un pronom"
        pro=""
        ref=""
    return pro,ref

### Traitement des GN

In [113]:
def faire_gn(departP,cas):
    global erg_genre, erg_nombre, abs_genre, abs_nombre
    if debug or 1: print "groupe departP :", departP
    groupe_nom=[]
    if departP[0]=="de" and departP[1]=="les":
        depart=["des"]+departP[2:]
    else:
        depart=departP
    if debug or 1: print "groupe depart :", depart
    groupe_nom.append(depart[0])
    if debug: print depart[0]
    for mot in depart[1:]:
        if debug: print mot
        if not "#" in mot:
            groupe_nom.extend(etendre_contraction([mot]))
    if debug: print "groupe nom :", groupe_nom
    mots=[]
    pro=[]
    det=[]
    adj=[]
    nom=[]
    gp=[]
    structureSyntagme={key:[] for key in syntagmes['GN']}
    tete=""
    nombre=""
    classe=""
    classesNom=[]
    reste=0
    groupe_nom=[element for element in groupe_nom if element!=""]
    if len(groupe_nom)==1:
#         print "Pronom ?",groupe_nom
        nomSeul=groupe_nom[0]
        boolPro=("#" in nomSeul)
    else:
        boolPro=False
    if boolPro:
        proForm,proRef=getRef(nomSeul)
        nomLexeme=PFM.lexique.formeLexeme[proForm][0]
        print "ref nomSeul",proForm,proRef
        nombre=getNounNumber(proRef,nombre="")
        classe,classesNom=getNounGender(proRef)
        pro.append(nomLexeme)
#                    if typeCas=="NoCas":
#                        cas=""
        if not "Cas" in boolAttribut["NOM"] or not boolAttribut["NOM"]["Cas"]:
            cas=""
#                    else:
        cellule=classe.capitalize()+nombre.capitalize()+cas.capitalize()
        cellules["PRO"].add(".".join([nomLexeme,classe.capitalize(),nombre.capitalize(),cas.capitalize()]).strip("."))
        if debug or 0:
            print mot,nomLexeme,nombre,cellule


        if cas=="ERG":
            erg_genre=classe
            erg_nombre=nombre
            if debug: print "ERG",erg_genre, erg_nombre
        elif cas=="ABS":
            abs_genre=classe
            abs_nombre=nombre
            if debug: print "ABS",abs_genre, abs_nombre

        
    else:
        for mot in groupe_nom:
            if reste==0:
    #            if mot=="deux" and "Du" in nombresNom:
                if mot=="deux" and "Du" in valAttributs["Nombre"]:
                    nombre="DU"
                    if det==[]: det.append(PFM.lexique.formeLexeme["des"][0])
                else:
                    if "Pau" in valAttributs["Nombre"] and mot in "deux trois quatre cinq".split():
                        nombre="PAU"
                        if det==[]: det.append(PFM.lexique.formeLexeme["des"][0])
                    if mot[len(mot)-1] in ["P","D"]:
                        nomLexeme=PFM.lexique.formeLexeme[mot[:-1]][0]
                    else:
                        try:
                            nomLexeme=PFM.lexique.formeLexeme[mot][0]
                        except:
                            nomLexeme=PFM.lexique.formeLexeme[mot.lower()][0]
                    categorie=PFM.lexique.lexemes[nomLexeme].classe.split(".")[-1]
                    if debug or 0: 
                        print "mot",[mot]
                        print "vedette",nomLexeme,categorie
                        print "categories",PFM.hierarchieCF.classes["NOM"],PFM.hierarchieCF.getCategory(categorie)
                    if PFM.hierarchieCF.getCategory(categorie)=="NOM":
                        tete=categorie
                        if debug or 0: print "tête :", tete
                        tampon=tete.split('.')
    #                    classe=tampon[0]
                        classesNom=nomLexeme.split('.')[1:]
                        if debug or 0: print "classesNom",classesNom 
                        classe=[classeElement for classeElement in classesNom if classeElement in gloses["NOM"]["Genre"]][0]
                        if debug or 0: print "classes",mot,classe,classesNom
                        try:
                            typeMot=tampon[1]
                        except IndexError:
                            typeMot=''
                        # modif pour les mots à pluriels en x
                        # 23/11/19
                        #
                        nombre=getNounNumber(mot,nombre)
#                         if mot[len(mot)-1] in ["s","x"]:
#                             if nombre=="": nombre="PL"
#                         elif mot[len(mot)-1] in ["P","D"]:
#                             if nombre=="":
#                                 # modif pour permettre le pluriel dans les cas où il n'y a pas de paucal.
#                                 # 30/5/21
#                                 #
#                                 if "Pau" in valAttributs["Nombre"]:
#                                     nombre="PAU"
#                                 elif "Du" in valAttributs["Nombre"] and mot[len(mot)-1]=="D":
#                                     nombre="DU"
#                                 else:
#                                     nombre="PL"
#                         else:
#                             if nombre=="": nombre="SG"
                        if debug or 0:
                            print mot,nomLexeme,nombre

                        nom.append(nomLexeme)
    #                    if typeCas=="NoCas":
    #                        cas=""
                        if not "Cas" in boolAttribut["NOM"] or not boolAttribut["NOM"]["Cas"]:
                            cas=""
    #                    else:
                        cellule=classe.capitalize()+nombre.capitalize()+cas.capitalize()
                        cellules["NOM"].add(".".join([nomLexeme,nombre.capitalize(),cas.capitalize()]).strip("."))
                        if debug or 0:
                            print mot,nomLexeme,nombre,cellule


                        if cas=="ERG":
                            erg_genre=classe
                            erg_nombre=nombre
                            if debug: print "ERG",erg_genre, erg_nombre
                        elif cas=="ABS":
                            abs_genre=classe
                            abs_nombre=nombre
                            if debug: print "ABS",abs_genre, abs_nombre
                    elif PFM.hierarchieCF.getCategory(categorie) in ["DET"]:
#
# Modif pour accepter les majuscules en début de phrase
#
#                         det.append(PFM.lexique.formeLexeme[mot][0])
                        det.append(nomLexeme)

    #                elif PFM.hierarchieCF.getCategory(categorie) in ["ADJ"] or ("ADJ" in PFM.hierarchieCF.classes and categorie in PFM.hierarchieCF.classes["ADJ"]):
                    elif PFM.hierarchieCF.getCategory(categorie) in ["ADJ"]:
#
# Modif pour accepter les majuscules en début de phrase
#
#                         adj.append(PFM.lexique.formeLexeme[mot][0])
                        adj.append(nomLexeme)

                    elif categorie=="PREP":			#Si on trouve une PREP, elle et le reste forment un GP
                        gp.append(mot)
                        reste=1
            else:							#On a trouvé une PREP, toute la suite va dans GP
                gp.append(mot)
    if debug: print "accord :", tete
    if reste==1: gp=faire_gp(gp)
    if debug: print "GP dans le GN : ", gp
    if debug: print "GN sans det ? ", det
    if not det and not boolPro: 
        if nom[0][0]==nom[0][0].upper():
            det.append(PFM.lexique.formeLexeme["les"][0])
        else:
            det.append(PFM.lexique.formeLexeme["des"][0])

    tempSyntagme=[]

    if pro:
        for mot in pro:
    #		glose=faire_glose(mot,classe,type,nombre)
            noligMot=mot.split(".")[0]
            for ligature in deligatures:
                noligMot=noligMot.replace(ligature,deligatures[ligature])
            #
            # traitement des noms propres avec tiret
            # traitement des noms paucals sans ordinaux
            #
            if "-" in noligMot:
                noLigs=noligMot.split("-")
                if len(noLigs)>1:
                    noligMot=noLigs[0]+"".join([c for c in noLigs[1:]])
            ref="\\"+recoder(noligMot,deaccent)+cellule
            print ref
    #        mots.append(ref)
            lexemesLocaux.add(noligMot)
            texte.append(ref)
            tempSyntagme.append(ref)
        structureSyntagme["NOM"]=tempSyntagme
        mots.insert(syntagmes['GN'].index("NOM"),structureSyntagme["NOM"])
        
    for mot in det:
        (casDet,nombreDet)=(cas,nombre)
        if not "Cas" in boolAttribut["DET"] or not boolAttribut["DET"]["Cas"]:
            casDet=""
        if not "Nombre" in boolAttribut["DET"] or not boolAttribut["DET"]["Nombre"]:
            print "pas de nombre"
            nombreDet=""
        if not "Genre" in boolAttribut["DET"] or not boolAttribut["DET"]["Genre"]:
            classeDet=""
        else:
#             print mot,casDet,nombreDet
            classeDet=[classeElement for classeElement in classesNom if classeElement in gloses["DET"]["Genre"]][0]
#		glose=faire_glose(mot,classe,type,nombre)

        cellules["DET"].add(".".join([mot,classeDet,nombreDet,casDet]).strip("."))


        noligMot=mot.split(".")[0]
        for ligature in deligatures:
            noligMot=noligMot.replace(ligature,deligatures[ligature])
        noligMot=noligMot.replace("-","")
        ref="\\"+recoder(noligMot,deaccent).upper()
        for attributDet in morphosyntax["Attributs"]["DET"]:
            if attributDet=="Cas":
                ref+=casDet.capitalize()
            elif attributDet=="Nombre":
                ref+=nombreDet.capitalize()
            elif attributDet=="Genre":
                ref+=classeDet.capitalize()
        tempSyntagme.append(ref)
        lexemesLocaux.add(noligMot)
        texte.append(ref)
    structureSyntagme["DET"]=tempSyntagme
    mots.insert(syntagmes['GN'].index("DET"),separateurMots.join(structureSyntagme["DET"]))
    tempSyntagme=[]
    for mot in gp: 
#        mots.append(mot)
        tempSyntagme.append(mot)
    structureSyntagme["GP"]=tempSyntagme
    mots.insert(syntagmes['GN'].index("GP"),structureSyntagme["GP"])
    tempSyntagme=[]
    for mot in adj:
#		glose=faire_glose(mot,classe,type,nombre)
        (casAdj,nombreAdj)=(cas,nombre)
        if not "Cas" in boolAttribut["ADJ"] or not boolAttribut["ADJ"]["Cas"]:
            casAdj=""
        if not "Nombre" in boolAttribut["ADJ"] or not boolAttribut["ADJ"]["Nombre"]:
            nombreAdj=""
        if not "Genre" in boolAttribut["ADJ"] or not boolAttribut["ADJ"]["Genre"]:
            classeAdj=""
        else:
            classeAdj=[classeElement for classeElement in classesNom if classeElement in gloses["ADJ"]["Genre"]][0]

        cellules["ADJ"].add(".".join([mot,classeAdj,nombreAdj,casAdj]).strip("."))

        noligMot=mot.split(".")[0]
        for ligature in deligatures:
            noligMot=noligMot.replace(ligature,deligatures[ligature])
        noligMot=noligMot.replace("-","")    
        ref="\\"+recoder(noligMot,deaccent).lower()+classeAdj.capitalize()+nombreAdj.capitalize()+casAdj.capitalize()
#        mots.append(ref)
        lexemesLocaux.add(noligMot)
        texte.append(ref)
        tempSyntagme.append(ref)
    structureSyntagme["GADJ"]=tempSyntagme
    mots.insert(syntagmes['GN'].index("GADJ"),structureSyntagme["GADJ"])
    tempSyntagme=[]
#    print "classe,nombre,cas", classe, nombre, cas
    for mot in nom:
#		glose=faire_glose(mot,classe,type,nombre)
        noligMot=mot.split(".")[0]
        for ligature in deligatures:
            noligMot=noligMot.replace(ligature,deligatures[ligature])
        #
        # traitement des noms propres avec tiret
        # traitement des noms paucals sans ordinaux
        #
        if "-" in noligMot:
            noLigs=noligMot.split("-")
            if len(noLigs)>1:
                noligMot=noLigs[0]+"".join([c.lower() for c in noLigs[1:]])
#         noligMot=noligMot.replace("-","")    
        if noligMot.istitle():
            ref="\\"+recoder(noligMot,deaccent)+cellule
        else:
            ref="\\"+recoder(noligMot,deaccent).lower()+cellule
#        mots.append(ref)
        lexemesLocaux.add(noligMot)
        texte.append(ref)
        tempSyntagme.append(ref)
    structureSyntagme["NOM"]=tempSyntagme
    mots.insert(syntagmes['GN'].index("NOM"),structureSyntagme["NOM"])
    listeMots=[]
    for element in syntagmes['GN']:
        listeMots+=structureSyntagme[element]
    if debug or 0:
        print "fin faire_gn",listeMots
    return (listeMots,(classesNom,nombre,cas))

### Traitement des GP

In [114]:
def multiplier_gp(groupe_prep,fonction=""):
    mots=[]
    lGroupes=[list(group) for k, group in groupby(groupe_prep, lambda x: x == ";") if not k]
    if len(lGroupes)>1:
        print lGroupes
    for lGp in lGroupes:
        mots.extend(faire_gp(lGp,fonction))
    return mots
    
def faire_gp(groupe_prep,fonction=""):
    mots=[]
    groupe_prep=etendre_contraction(groupe_prep)
    if debug or 1: print "faire_gp", " ".join(groupe_prep), "fonction",fonction
    formePreposition=groupe_prep[0]
    preposition=PFM.lexique.formeLexeme[formePreposition][0]
    if debug or 1: print formePreposition, preposition
#    if preposition!="à" or typeCas=="NoCas":
    if not "Cas" in boolAttribut["NOM"] or not boolAttribut["NOM"]["Cas"] or fonction!="IND":
        if debug: print u"fonction!=IND",[groupe_prep[0],u"à"]

        # noligMot=groupe_prep[0]
        noligMot=preposition
        for ligature in deligatures:
            noligMot=noligMot.replace(ligature,deligatures[ligature])
        noligMot=noligMot.replace("-","")
        ref="\\"+recoder(noligMot,deaccent).upper()
        
        if debug: print ref
        if casSyntagmes and preposition in casPreposition:
            if debug: 
                print casPreposition, casPreposition[preposition]
            cas=casPreposition[preposition]            
            if "+" in casPreposition[preposition]:
                cas=cas.strip("+")
                mots.append(ref)
                lexemesLocaux.add(noligMot.upper())
                texte.append(ref)
        else:
            if debug or 1: print "pas de Cas"
            cas=""
            mots.append(ref)
            lexemesLocaux.add(noligMot.upper())
            texte.append(ref)
        if debug or 1:
            print "groupe prep :", groupe_prep
            print "ref",ref,noligMot
        if len(groupe_prep)>1:
            groupe_nom=groupe_prep[1:]
            if debug: print groupe_nom[0]
            #
            # Choisir le cas en fonction de la préposition
            #
            (localMots,(localClasses,localNombre,localCas))=faire_gn(groupe_nom,cas)
            mots.insert(syntagmes['GP'].index("GN"),localMots)
        if debug: 
            print groupe_prep, cas, mots
        return mots
    elif fonction=="IND":
        groupe_nom=groupe_prep[1:]
        cas=casSyntagmes["IND"]
        if "+" in casSyntagmes["IND"]:
            
            noligMot=groupe_prep[0]
            for ligature in deligatures:
                noligMot=noligMot.replace(ligature,deligatures[ligature])
            
            ref="\\"+recoder(noligMot,deaccent).upper()
            cas=cas.strip("+")
            mots.append(ref)
            lexemesLocaux.add(noligMot.upper())
            texte.append(ref)
        if debug: print "faire_gn", groupe_nom, faire_gn(groupe_nom,cas)
        (localMots,(localClasses,localNombre,localCas))=faire_gn(groupe_nom,cas)
        mots.append(localMots)
        return mots

### Traitement des contractions

In [115]:
def etendre_contraction(liste):
    result=[]
    if liste[0] in contractions.keys():
        if debug: print "EXT : ", liste, contractions[liste[0]],liste[1:] 
        result.extend(contractions[liste[0]])
        result.extend(liste[1:])
    else:
        result=liste
    return result

In [116]:
etendre_contraction("une jeune princesse mariée".split(" "))

['une', 'jeune', 'princesse', 'mari\xc3\xa9e']

In [117]:
def printflat(liste,suffixe="",prefixe=""):
    if debug: print "printflat", liste
    if not isinstance(liste, basestring):
        for element in liste:
            accumulerMots(prefixe)
            printflat(element,suffixe)
    else:
#        if "{preview}" in liste:
#            print "preview",prefixe,liste,suffixe
        accumulerMots(prefixe)
        accumulerMots(liste+suffixe)

# Traitements

In [118]:
#######################
#
#	INITIALISATION DES VARIABLES
#
#######################

try:
    __IPYTHON__ 
    ipython=True
except: 
    ipython=False

In [119]:
################
#
# LECTURE DU FICHIER DE LEXEMES
#
#		LES LIGNES QUI COMMENCENT PAR # SONT IGNOREES
#
################
texHeader=[]
if "theNotebook" in globals():
    version=theNotebook
    texHeader.append("%% script : "+version)
texHeader.append('%%%% run : %s' % time.strftime("%y%m%d-%H%M"))

if ipython or True:
#    lexeme_nom=serie+"Lexemes.txt"
#    phrase_nom=serie+"Phrases.txt"
    phrase_nom=serie+complementPhrases+"Phrases.csv"
    traduction_nom=serie+complementPhrases+"Traductions.csv"
    ecriture_nom=serie+complementPhrases+"Ecrit.csv"
    ordre_nom=serie+complementPhrases+"Ordre.csv"
    mots_nom=serie+complementPhrases+"Mots.csv"
    cloze_nom=serie+complementPhrases+"Clozes.txt"
else:
    parser=optparse.OptionParser()
    parser.add_option("-o", "--out", dest="outfile", action="store_true", help="write to FILE")
    parser.add_option("-c", "--cloze", dest="print_cloze", action="store_true", help="write a CLOZE FILE")
    parser.add_option("-l", "--lexicon", dest="print_lexique", action="store_true", help="append a lexicon")
    parser.add_option("-r", "--roots", dest="print_racines", action="store_true", help="append a root list")

    (options, args) = parser.parse_args()
    lexeme_nom=args[0]
    phrase_nom=args[1]
    if len(args)>=3:
        traduction_nom=args[2]
    else:
        traduction_nom=""
    if len(args)>=4:
        ecriture_nom=args[3]
    else:
        ecriture_nom=""
    if len(args)>=5:
        ordre_nom=args[4]
    else:
        ordre_nom=""
    if len(args)>=6:
        mots_nom=args[5]
    else:
        mots_nom=""


#### Ouverture du fichier lexique

In [120]:
def filtrerCloze(filtre):
    result=[]
    for ligne in clozeLines:
        if type(ligne)==str:
            ligne=unicode(ligne.decode('utf8'))
        ligne=ligne.strip()
        if not ligne.startswith("#"):
            clozeElements=ligne.split(";")
            if clozeElements[1] in filtre:
                result.append(ligne)
        else:
            result.append(ligne)
    return result

In [121]:
def flatten(lis):
     for item in lis:
         if not isinstance(item, basestring):
             for x in flatten(item):
                 yield x
         else:        
             yield item

In [122]:
previewerBegin="\\begin{preview}\\begin{flushleft}"
previewerEnd="\\end{flushleft}\\end{preview}"

def latex2ipa(mot):
    debutLignePreview="\\begin{preview}"
    finLignePreview="\\end{preview}"
    if " " in mot:
        result=[]
        for element in mot.split():
            result.append(latex2ipa(element))
        return separateurMots.join(result)
    else:
        mot=mot.replace(previewerBegin,"").replace(previewerEnd,"")
    #    print mot
        mot=mot.replace(debutLignePreview,"").replace(finLignePreview,"")
    #    print mot
        element=mot.replace("P{}"," ").replace("{}"," ")
    #    print element
        cleElement=element.strip().strip("\\{}")
    #    print cleElement
        if element=="\\ex": cleElement=""
        if cleElement:
            if not cleElement in traductions:
                print mot
                print cleElement
                print traductions
                warnings.warn("pas de transcription pour %s"%cleElement)
                return None
            else:
                return traductions[cleElement]
        else:
            return None

In [123]:
#latex2ipa("\\begin{preview}\\DEFMSg{}")

In [124]:
def previewer(chaine,numLignesVides=0,suffixe=""):
    flatChaine=[mot+suffixe for mot in list(flatten(chaine))]
    result=previewerBegin+separateurMots.join(flatChaine)+"\\\\"*numLignesVides+previewerEnd
    return [result]

In [125]:
def recoder(chaine,table):
    if type(chaine)==str:
        temp=unicode(chaine.decode('utf8')).translate(table)
        result=temp.encode('utf8')
    elif type(chaine)==unicode:
        result=chaine.translate(table)
    return result

In [126]:
accentedIn = unicode(phonology["translations"]["deaccent"]["in"])
deaccentIn = [ord(char) for char in accentedIn]
deaccentOut = unicode(phonology["translations"]["deaccent"]["out"])
deaccent = dict(zip(deaccentIn, deaccentOut))

In [127]:
tipaIn = unicode(phonology["translations"]["ipa"]["in"])
ipaIn = [ord(char) for char in tipaIn]
ipaOut = unicode(phonology["translations"]["ipa"]["out"])
toipa = dict(zip(ipaIn, ipaOut))

In [128]:
#################################################
#################################################
#################################################
##
##
##	FAIRE LE TRI DES FORMES UTILISEES DANS LES PHRASES
##	AFFICHER DANS LES TABLEAUX SEULEMENT CES FORMES
##
##
#################################################
################################################
#
#
#	FAIRE LA LISTE DES PHRASES AVEC LES 4 LIGNES
#		GRAPHO, PHONO, GLOSE, TRAD
#
#
################################################
texte=[]
texteMots=set()
#graphies={}
#abs_genre=""
#abs_nombre=""
#PFM.lexique.formeLexeme

In [129]:
def normaliserMajusculesMinuscules(mot):
    result=mot
    if "'" in mot:
        lMots=mot.split("'")
        result="'".join([normaliserMajusculesMinuscules(m) for m in lMots])
    elif len(mot)>1 and mot!=mot.lower() and mot!=mot.capitalize():
        result=mot[0]+mot[1:].lower()
    return result

In [130]:
normaliserMajusculesMinuscules(u"d'E-St'OGM")

u"d'E-st'Ogm"

### Introduire les pronoms dans les phrases

- clitiques verbaux entre dièses pour permettre l'inclusion en français et l'exclusion en kalaba
  - Marie TAB #les# voyait TAB ,les enfants TAB TAB dans la cour
    - FR: Les enfants, Marie les voyait dans la cour.
    - KB: Marie voyait les enfants dans la cour.

In [131]:
def cleanFr(line):
    result=line
    m=re.search(ur"((\w+)#\S+#)",result)
    if m:
        result=result.replace(m.group(1),m.group(2))
        result=cleanFr(result)
    return result
    

In [132]:
cleanFr(u"elles#sorcières# pensent à lui#garçon#")

u'elles pensent \xe0 lui'

## Assemblage des phrases

### Niveau phrase

In [133]:
def faire_phrases(phrase_file,sortie="latex"):
    print sortie
    phrasesLocales=phrase_file.readlines()
    syntagmesLocaux=[]
    numLignes=len(phrasesLocales)
    numImages=1
    numLignesVides=0
    if sortie in ["latex","traductions"]:
        finLigne="\\\\"
    elif sortie=="images":
        numImages=1
        numLignesVides=0
    else:
        finLigne="\n"
        numLignesVides=0

    debutLignePreview="\\begin{preview}"
    finLigneNoPreview="\\\\"*numLignesVides
    finLignePreview="\\\\"*numLignesVides+"\\end{preview}"
    numLignes=(len(phrasesLocales)//numImages)*numImages

    if print_phrases:
        comment=""
    else:
        comment="%"
    if sortie in ["latex","traductions"]: ajouterExemple("\\begin{phrases}")
    for nPhrase,line in enumerate(phrasesLocales[:numLignes]):
        if debug: print nPhrase,line
        #phrase contient une position par fonction
        #pour accueillir les équivalents kalabas des chunks français
        phrase=[0 for i in range(len(syntagmes['Phrase']))]
        tampon=(line.strip().rstrip('.')).replace("'"," ").replace(u"’"," ").split("\t")
        tampon=[t.strip() for t in tampon]
    #    print tampon[0],type(tampon[0])
        if debug or 1: 
            print "==========================================="            
            print line
        if not tampon[0].startswith("#"):
#             print nPhrase,tampon
            #
            # Introduire les clitiques non-traités
            # 
            # #les# voit 
            #
            m=re.search(ur"#.*#\s*(.*)",tampon[1])
            if m:
                verbe=m.group(1).split(" ")
            else:
                verbe=tampon[1].split(" ")
            verbeForme=verbe[0]
            if len(PFM.lexique.formeLexeme[verbeForme])!=1:
                print "FORME AMBIGUË", PFM.lexique.formeLexeme[verbeForme]
            verbeLexeme=PFM.lexique.formeLexeme[verbeForme][0]
            tempVerbe=verbeLexeme.split(".")
            if debug or 0: print tempVerbe
            formeCitation=tempVerbe[0]
            lexemesLocaux.add(formeCitation)
            for element in tempVerbe[1:]:
                if element in attributsFlexVerbe:
                    formeCitation+=element.capitalize()
            if len(tempVerbe)>1:
                typeVerbe=tempVerbe[1]
            else: 
                typeVerbe=""
            classeVerbe=""
            nombreVerbe=""
            nombrePersonne=""
    #        print verbeLexeme, formeCitation,typeVerbe
            verbeLemme="%s%s"%(formeCitation,typeVerbe.capitalize())
            verbeFormeIndex=PFM.lexique.lexemes[verbeLexeme].formes.index(verbeForme)
            if debug: print "verbe :", verbe
    #        if verbeLexeme.endswith("VI"):
            if casSyntagmes and "SUJ" in casSyntagmes and "Erg" in casSyntagmes["SUJ"]:
                if ".VI" in verbeLexeme:            
    #                suj_cas="Abs"
                    frSujetCas="Abs"
                    frObjetCas=""
                    klbSujet="SUJ"
                else:
    #                suj_cas="Erg"
    #                obj_cas="Abs"
                    frSujetCas="Erg"
                    frObjetCas="Abs"
                    klbSujet="OBJ"
            elif casSyntagmes and "SUJ" in casSyntagmes and "Nom" in casSyntagmes["SUJ"]:
                frSujetCas=casSyntagmes["SUJ"]
                frObjetCas=casSyntagmes["OBJ"]
                klbSujet="SUJ"
            else:
                frSujetCas=""
                frObjetCas=""
                klbSujet="SUJ"
            if debug: print (frSujetCas,frObjetCas,klbSujet)
            suj_genre=valAttributs["Genre"][0]
            suj_nombre=valAttributs["Nombre"][0]
            obj_genre=valAttributs["Genre"][0]
            obj_nombre=valAttributs["Nombre"][0]
            abs_genre=valAttributs["Genre"][0]
            abs_nombre=valAttributs["Nombre"][0]
            sujet=tampon[0].strip().split(" ")
            (localMots,(localClasses,localNombre,localCas))=faire_gn(sujet,frSujetCas)
            if (debug or 0) and nPhrase==0: print ("FAIRE_GN SUJET",localMots,(localClasses,localNombre,localCas))
            if sortie=="ordre":
                phrase[syntagmes['Phrase'].index('SUJ')]=previewer(localMots,suffixe="{}")
            else:
                phrase[syntagmes['Phrase'].index('SUJ')]=localMots
            if debug: print localCas,localNombre,localClasses,sujetVerbe
            if not localCas or localCas in sujetVerbe:
                if localClasses and "Genre" in gloses["VER"]:
                    localClasse=[classeElement for classeElement in localClasses if classeElement in gloses["VER"]["Genre"]][0]
                else:
                    localClasse=""
                classeVerbe=localClasse
                nombreVerbe=localNombre
                nombrePersonne=localNombre
                casVerbe=localCas
                if (debug or 0) and nPhrase==0: print "frSuj",nombrePersonne
            if debug: print "sujet :",phrase[1]
            ########################
            #
            # Modification pour permettre les accords par défaut
            # avec des verbes transitifs sans objet pour les systèmes ergatifs
            #
            ########################
            if klbSujet=="OBJ" and len(tampon)<3:
                tampon.append("")
            if len(tampon)>=3:
                if tampon[2] and tampon[2].startswith(","):
                    tampon[2]=tampon[2][1:]
                objet=tampon[2].split(" ")
                if debug: print "objet : ",objet
                if objet!=['']: 
                    (localMots,(localClasses,localNombre,localCas))=faire_gn(objet,frObjetCas)
                    if (debug or 0) and nPhrase==0: print ("frObj",(localClasses,localNombre,localCas),line)
                    if sortie=="ordre":
                        phrase[syntagmes['Phrase'].index('OBJ')]=previewer(localMots,suffixe="{}")
                    # pourquoi un traitement spécial pour sortie=="mots"
#                     elif sortie=="mots": print "objet",objet
                    else:
                        phrase[syntagmes['Phrase'].index('OBJ')]=localMots
                    if debug or 0: print ("obj :",localCas, sujetVerbe)
                    if localCas in sujetVerbe and localCas!="Nom":
                        if localClasses:
                            localClasse=[classeElement for classeElement in localClasses if classeElement in gloses["VER"]["Genre"]][0]
                        else:
                            localClasse=""
                        classeVerbe=localClasse
                        nombreVerbe=localNombre
                        nombrePersonne=localNombre
                        casVerbe=localCas
                        if (debug or 0) and nPhrase==0: print ("cas",nombrePersonne, line)
                elif klbSujet=="OBJ":
                    print "verbe ergatif sans objet",[objet]
                    print tampon
                    if boolAttribut["VER"]["Genre"]:
                        classeVerbe=morphosyntax["Defauts"]["Genre"]
                    if boolAttribut["VER"]["Nombre"]:
                        nombreVerbe=morphosyntax["Defauts"]["Nombre"]
                    if boolAttribut["VER"]["Pers"]:
                        nombrePersonne=morphosyntax["Defauts"]["NbPers"]
                    
            if len(tampon)>=4:
                if tampon[3] and tampon[3].startswith(","):
                    tampon[3]=tampon[3][1:]
                indirect=tampon[3].split(" ")
                if debug: print "indirect : ",indirect
                if indirect!=['']:
                    if sortie=="ordre":
                        phrase[syntagmes['Phrase'].index('IND')]=previewer(faire_gp(indirect,"IND"),suffixe="{}")
                    else:
                        phrase[syntagmes['Phrase'].index('IND')]=faire_gp(indirect,"IND")
            ###################
            #
            # Modification pour ajouts multiples séparés par ;
            # 24/11/19
            #
            ###################
            if len(tampon)>=5:
                if tampon[4] and tampon[4].startswith(","):
                    tampon[4]=tampon[4][1:]
                comp=tampon[4].split(" ")
                if comp!=['']:
                    if sortie=="ordre":
                        phrase[syntagmes['Phrase'].index('COMP')]=previewer(multiplier_gp(comp),suffixe="{}")
                    else:
                        phrase[syntagmes['Phrase'].index('COMP')]=multiplier_gp(comp)
            if len(tampon)>=6:
                if tampon[5] and tampon[5].startswith(","):
                    tampon[5]=tampon[5][1:]
                ajout=tampon[5].split(" ")
                if sortie=="ordre":
                    phrase[syntagmes['Phrase'].index('AJOUT')]=previewer(multiplier_gp(ajout),suffixe="{}")
                else:
                    phrase[syntagmes['Phrase'].index('AJOUT')]=multiplier_gp(ajout)
            if not boolAttribut["VER"]["Genre"]:
                classeVerbe=""
            if not boolAttribut["VER"]["Nombre"]:
                nombreVerbe=""
            if not boolAttribut["VER"]["Pers"]:
                nombrePersonne=""
            else:
                nombrePersonne="Trois"+nombrePersonne.capitalize()
            if (debug or 0) and nPhrase==0: print "accord Verbe",nombreVerbe, classeVerbe, sujetVerbe, line

            noligFormeCitation=formeCitation
            for ligature in deligatures:
                noligFormeCitation=noligFormeCitation.replace(ligature,deligatures[ligature])
            noligFormeCitation=noligFormeCitation.replace("-","")
            
            glose="\\"+recoder(noligFormeCitation,deaccent)\
                +morphosyntax["VER"]["FormesBase"][verbeFormeIndex].capitalize()\
                +nombrePersonne\
                +classeVerbe.capitalize()\
                +nombreVerbe.capitalize()
            cellules["VER"].add(".".join([verbeLexeme,morphosyntax["VER"]["FormesBase"][verbeFormeIndex].capitalize(),nombrePersonne,classeVerbe,nombreVerbe]).strip("."))
            if (debug or 0) and nPhrase<200: print "glose Verbe",glose
            if sortie=="ordre":
                phrase[syntagmes['Phrase'].index('VER')]=previewer([glose],suffixe="{}")
            else:
                phrase[syntagmes['Phrase'].index('VER')]=glose
            lexemesLocaux.add(noligFormeCitation)
            texte.append(glose)
            if sortie in ["latex","traductions"]: 
                ajouterExemple("\\ex")
                if print_glose and print_phono and print_ortho:
                    ajouterExemple(comment+"\\gloses")
                elif print_glose+print_ortho+print_phono==2:
                    ajouterExemple(comment+"\\gloses")
            syntagmesBruts=[]
            syntagmesEtiquetes=[]
            for nMot,mot in enumerate(phrase):
                if mot!=0:
                    
                    if isinstance(mot,basestring):
                        syntagmeLocal=latex2ipa(mot)
                    else:
                        interFlat=[]
                        for m in flatten(mot):
                            mList=re.findall(ur"\\[^{]+{}",m)
                            if mList:
                                interFlat.extend(mList)
                            else:
                                interFlat.append(m)
                        if debug: print "interFlat",interFlat
                        if "RightLeft" in globals() and RightLeft and sortie=="latex":
                            interFlat.reverse()
                        syntagmeLocal=separateurMots.join([latex2ipa(f) for f in interFlat])
                    syntagmesBruts.append(syntagmeLocal)
                    syntagmesEtiquetes.append(syntagmeLocal)
                    if "RightLeft" in globals() and RightLeft and sortie=="latex":
                        syntagmesEtiquetes.append(syntagmes["Phrase"][nMot])
                    else:
                        syntagmesEtiquetes.insert(-1,syntagmes["Phrase"][nMot])
#                    syntagmesEtiquetes.insert(-1,syntagmes["Phrase"][nMot])
                    printflat(mot,"{}")
            if "RightLeft" in globals() and RightLeft and sortie=="latex":
                ligneSyntagmesLocaux=separateurMots.join(syntagmesBruts[::-1])+";"+" ".join(syntagmesEtiquetes[::-1])
            else:
                ligneSyntagmesLocaux=separateurMots.join(syntagmesBruts)+";"+" ".join(syntagmesEtiquetes)
#            ligneSyntagmesLocaux=separateurMots.join(syntagmesBruts)+";"+" ".join(syntagmesEtiquetes)
#            print ligneSyntagmesLocaux
            syntagmesLocaux.append(ligneSyntagmesLocaux)
            localAccumulateur=[accu for accu in accumulateur if accu!=""]
            if print_ortho:
                prefixe=""
                if sortie=="images":
                    if nPhrase%numImages==0:
                        prefixe="\\begin{preview}"
                    else:
                        prefixe=""
                    if nPhrase%numImages==numImages-1:
                        finLigne=finLignePreview
                    else:
                        finLigne=finLigneNoPreview
                elif sortie=="mots":
                    localAccumulateur=[debutLignePreview+accu+finLignePreview for accu in accumulateur if accu!=""]
            else:
                prefixe=marqueurCommentaire
            ajouterExemple(prefixe+separateurMots.join(localAccumulateur)+finLigne)
            for mot in phrase:
                if mot!=0:
                    printflat(mot,"P{}")
            if print_phono:
                prefixe=""
                if sortie in ["images","ordre"]:
                    prefixe=marqueurCommentaire
            else:
                prefixe=marqueurCommentaire
            if "RightLeft" in globals() and RightLeft and sortie=="latex" and separateurMots=="":
                ajouterExemple(prefixe+separateurMots.join(accumulateur[::-1])+finLigne)
            else:
                ajouterExemple(prefixe+separateurMots.join(accumulateur)+finLigne)
            for mot in phrase:
                if mot!=0 and sortie in ["latex","traductions"]:
                    printflat(mot,"G{}")
            if print_glose:
                prefixe=""
                if sortie in ["images","ordre"]:
                    prefixe=marqueurCommentaire
            else:
                prefixe=marqueurCommentaire
            if sortie in ["latex","traductions"]: ajouterExemple(prefixe+separateurMots.join(accumulateur)+finLigne)
            ##############
            #
            # Modification pour ajouts multiples
            # suppression des ; de séparation dans la traduction
            #
            ##############
            if "," in line:
                consts=line.split("\t")
                disloc=[]
                canon=[]
                for const in consts:
                    if const.startswith(","):
                        if ";" in const:
                            subConsts=const.split(";")
                            print "############## ; ##########@"
                            print subConsts
                            disloc.append(subConsts[0][1:])
                            for subConst in subConsts[1:]:
                                canon.append(subConst.lstrip())
                        else:
                            disloc.append(const[1:])
                    else:
                        canon.append(const)
                line=",\t".join(disloc).strip()+", "+"\t".join(canon)
            if " ; " in line:
                line=line.replace(" ; "," ")
            line=cleanFr(line)            
            traduction=(line.strip().rstrip('.')).split()
            start=1
            for element in traduction:			# convertir les S majuscules à la finale des mots en minuscules
                if element==u"À":element=u"à"
                if element=="Au":element="au"
                if element=="Aux":element="aux"
                element=element.strip("#").replace(u"’","'")
                #
                # Modification pour accepter les suffixes P et D à la fin des noms
                # 31/5/21
                #
                if element[-2:] in ["sP","sD"]:
                    element=element[:-1]
                if element!="":
                    if start:
                        start=0
                        element=element.capitalize()
                    caracteres=normaliserMajusculesMinuscules(element)
#                    accumulerMots("".join(caracteres).encode('utf8'))
                    accumulerMots(caracteres)
            if sortie=="latex": 
                ajouterExemple(taches()+" ".join(accumulateur)+".")
            elif sortie=="images":
                ajouterExemple(marqueurCommentaire+"\\begin{preview}"+" ".join(accumulateur)+"."+"\\end{preview}")
            elif sortie in ["ordre","mots"]:
                ajouterExemple("\\begin{preview}"+" ".join(accumulateur)+"."+"\\end{preview}")
                #if sortie=="mots": ajouterExemple("\\begin{preview}"+"."+"\\end{preview}")
            else:
                if debug: print accumulateur
                ajouterExemple(" ".join(accumulateur)+".")
            del accumulateur[:]
            if sortie=="latex" and print_coffee and random.randint(1,4)==1:
                stain=random.choice(["A","B","C","D"])
                stain=random.choice(["A","B","C"])
                alpha=random.random()/1.5
                angle=random.randint(0,360)
                xoff=random.randint(-200,0)
#                ajouterExemple('\\\\\\cofe%sm{%.3f}{1}{%d}{%d}{0}' % (stain,alpha,angle,xoff))
                ajouterExemple('\\hspace{-.35\\textwidth}\\cofe%sm{%.3f}{1}{%d}{%d}{0}' % (stain,alpha,angle,xoff))

    if sortie in ["latex","traductions"]: ajouterExemple("\\end{phrases}")
    if sortie in ["latex","traductions"]:
        if ('options' in globals() and options.print_cloze) or print_lexique:
            tab=(head,tail,"")
        else:
            tab=(head,tail,"%")
#        prononciationExtrait=[]
        ajouterVocabulaire(tab[2]+"\\begin{itemize}")
        ajouterVocabulaire(tab[2]+"\\Needspace{8\\baselineskip}%")
        ajouterVocabulaire(tab[2]+"\\item NOMS\\\\[-3ex]")
        print_tableaux(dimensionsTableaux["NOM"]["cols"],tableaux["NOM"],texte,dimensionsTableaux["NOM"]["long"],tab,False)
        ajouterVocabulaire(tab[2]+"\\Needspace{8\\baselineskip}%")
        ajouterVocabulaire(tab[2]+"\\item ADJECTIFS\\\\[-3ex]")
        print_tableaux(dimensionsTableaux["ADJ"]["cols"],tableaux["ADJ"],texte,dimensionsTableaux["ADJ"]["long"],tab,False)
        ajouterVocabulaire(tab[2]+"\\Needspace{8\\baselineskip}%")
        ajouterVocabulaire(tab[2]+"\\item VERBES\\\\[-3ex]")
        print_tableaux(dimensionsTableaux["VER"]["cols"],tableaux["VER"],texte,dimensionsTableaux["VER"]["long"],tab,False)
#        print_tableaux(2,tableaux["VER"],texte,20,tab)
        ajouterVocabulaire(tab[2]+"\\Needspace{8\\baselineskip}%")
        ajouterVocabulaire(tab[2]+u"\\item DÉTERMINANTS\\\\[-3ex]")
        print_tableaux(dimensionsTableaux["DET"]["cols"],tableaux["DET"],texte,dimensionsTableaux["DET"]["long"],tab,False)
#        print_tableaux(3,tableaux["DET"],texte,0,tab)
        ajouterVocabulaire(tab[2]+"\\Needspace{8\\baselineskip}%")
        ajouterVocabulaire(tab[2]+u"\\item PRONOMS\\\\[-3ex]")
        print_tableaux(dimensionsTableaux["PRO"]["cols"],tableaux["PRO"],texte,dimensionsTableaux["PRO"]["long"],tab,False)
#        print_tableaux(3,tableaux["PRO"],texte,0,tab)
        ajouterVocabulaire(tab[2]+"\\Needspace{8\\baselineskip}%")
        ajouterVocabulaire(tab[2]+u"\\item PRÉPOSITIONS\\\\[-3ex]")
        print_tableaux(dimensionsTableaux["PREP"]["cols"],tableaux["PREP"],texte,dimensionsTableaux["PREP"]["long"],tab,False)
#        print_tableaux(2,tableaux["PREP"],texte,0,tab)
        ajouterVocabulaire(tab[2]+"\\end{itemize}")

        if sortie=="latex":
            with codecs.open(serie+complementPhrases+"Exemples"+variante+".tex", 'wb',encoding="utf8") as output:
                for texHead in texHeader:
                    output.write(texHead+"\n")
                for exemple in exemples:
                    output.write(exemple+"\n")
            with codecs.open(serie+complementPhrases+"Syntagmes"+variante+".txt", 'wb',encoding="utf8") as output:
                for exemple in syntagmesLocaux:
                    output.write(exemple+"\n")

        elif (sortie=="traductions" and variante=="-Corr"):
            with codecs.open(serie+complementPhrases+"Traductions"+variante+".tex", 'wb',encoding="utf8") as output:
                for texHead in texHeader:
                    output.write(texHead+"\n")
                for exemple in exemples:
                    output.write(exemple+"\n")            

        if sortie=="latex":
            with codecs.open(serie+complementPhrases+"Vocabulaire"+variante+".tex", 'wb',encoding="utf8") as output:
                for texHead in texHeader:
                    output.write(texHead+"\n")
                for vocable in vocabulaire:
    #                print [vocable]
                    output.write(vocable+"\n")

            with codecs.open(serie+complementPhrases+"Prononciation"+variante+".tex", 'wb',encoding="utf8") as output:
                for texHead in texHeader:
                    output.write(texHead+"\n")
                for ligne in prononciationBegin+prononciationExtrait+prononciationEnd:
                    output.write(ligne+"\n")
#    elif sortie=="images":
#        with codecs.open(serie+complementPhrases+"Images"+variante+".tex", 'wb',encoding="utf8") as output:
#            for exemple in exemples:
#                output.write(exemple+"\n")
    elif sortie in ["images","ordre","mots"]:
        with codecs.open(serie+complementPhrases+sortie.capitalize()+variante+".tex", 'wb',encoding="utf8") as output:
            for texHead in texHeader:
                output.write(texHead+"\n")
            for exemple in exemples:
                output.write(exemple+"\n")
    if sortie in ["images","traductions","ordre","mots"]:
        if sortie=="mots":print
        clozeTraductions=[]
        clozeExemples=exemples[:]
        filtreLignes=["\\begin{phrases}","\\ex","\\gloses","\\end{phrases}"]
        if sortie=="traductions":
            clozeExemples=[exemple for exemple in clozeExemples if (not exemple in filtreLignes) and not exemple.startswith("%")]
        for orthoLigne,phonoLigne,tradLigne in zip(*[iter(clozeExemples)]*3):
            if sortie=="traductions" and 0:
                print "ortho",orthoLigne
                print "phono",phonoLigne
                print "trad",tradLigne
            phonoLigne=phonoLigne.replace(previewerBegin,"").replace(previewerEnd,"")
            phonoLigne=phonoLigne.replace("P{}"," ").replace("{}"," ")
            phonoLigne=phonoLigne.replace(debutLignePreview,"").replace(finLignePreview,"")
            phonoMots=phonoLigne.strip(marqueurCommentaire).replace(finLignePreview,"").strip().split()
            tradLigne=tradLigne.strip(marqueurCommentaire).replace(debutLignePreview,"").replace(finLignePreview,"")
            phonoPhrase=[]
            for element in phonoMots:
                cleElement=element.strip().strip("\\{}")
                if element=="\\ex": cleElement=""
                if cleElement:
                    if not cleElement in traductions: print "no key", cleElement,phonoMots
                    phonoPhrase.append(traductions[cleElement])
            if separateurPhonoCloze=="" : suppLigne=";"+" ".join(phonoPhrase)
            else: suppLigne=""
            result=separateurPhonoCloze.join(phonoPhrase)+";"+tradLigne+suppLigne
            clozeTraductions.append(result)
        with codecs.open(serie+complementPhrases+sortie.capitalize()+".txt", 'wb',encoding="utf8") as output:
            for ligne in clozeTraductions:
                output.write(ligne+"\n")
        if sortie=="ordre" and separateurMots=="":
            dictSyntagmes={}
            dictTraductions={}
            ordreCles=[]
            for element in syntagmesLocaux:
                clePhono,valSyntagmes=element.split(";")
                dictSyntagmes[clePhono]=" ".join([v for v in valSyntagmes.split() if not v in nomFonctions])
                ordreCles.append(clePhono)
            for element in clozeTraductions:
                clePhono,valTrad,valMots=element.split(";")
                dictTraductions[clePhono]=valTrad
            morceauxPhrases=["%s;%s"%(dictSyntagmes[c],dictTraductions[c]) for c in ordreCles]
            with codecs.open(serie+complementPhrases+"OrdreSyntagmes"+variante+".txt", 'wb',encoding="utf8") as output:
                for exemple in morceauxPhrases:
                    output.write(exemple+"\n")
                

### Traitement du fichier phrases

In [134]:
nomTableaux="Tableaux.yaml"
with open(serie+nomTableaux, 'r') as stream:
    tableaux=yaml.safe_load(stream)

In [135]:
sortie="traductions"
syntagmes=syntagmesLR
print syntagmes

lexemesLocaux=set()
if traduction_nom.endswith("csv") and variante!="-Corr":
    try:
        traduction_file = codecs.open(traduction_nom,"r","utf8")
    except IOError:
        print 'I could not open the translation file', traduction_nom
        sys.exit()      
    else:
        try:
            cloze_file = codecs.open(cloze_nom,"r","utf8")
        except IOError:
            print 'I could not open the cloze file', cloze_nom
            sys.exit()
        else:
            traductions={}
            for line in cloze_file.readlines():
                line=line.strip()
                if not line.startswith("#"):
                    elementsCloze=line.split(";")
                    traductions[elementsCloze[0]]=elementsCloze[5]
            cloze_file.close()
            exemples=[]
            accumulateur=[]
            vocabulaire=[]
#            prononciationExtrait=[]
            faire_phrases(traduction_file,sortie=sortie)
            traduction_file.close()

    localClozes=filtrerCloze(lexemesLocaux)
    with codecs.open(serie+complementPhrases+"%s-Clozes"%sortie.capitalize()+".txt", 'wb',encoding="utf8") as output:
        for ligne in localClozes:
            output.write(ligne+"\n")

{'Phrase': ['COMP', 'VER', 'AJOUT', 'SUJ', 'OBJ', 'IND'], 'GP': ['PREP', 'GN'], 'GADJ': ['ADV', 'ADJ', 'GP'], 'GN': ['GP', 'NOM', 'DET', 'GADJ']}


In [136]:
sortie="latex"
if "RightLeft" in globals() and RightLeft:
    syntagmes=syntagmesRL
else:
    syntagmes=syntagmesLR
print syntagmes

lexemesLocaux=set()
try:
    phrase_file = codecs.open((phrase_nom),"r","utf8")
except IOError:
    print 'I could not open the sentence file', phrase_nom
    sys.exit()
exemples=[]
accumulateur=[]
vocabulaire=[]
prononciationExtrait=[]
faire_phrases(phrase_file,sortie=sortie)
phrase_file.close()

localClozes=filtrerCloze(lexemesLocaux)
with codecs.open(serie+complementPhrases+"Phrases-Clozes"+".txt", 'wb',encoding="utf8") as output:
    for ligne in localClozes:
        output.write(ligne+"\n")

{'Phrase': ['IND', 'OBJ', 'SUJ', 'AJOUT', 'VER', 'COMP'], 'GP': ['GN', 'PREP'], 'GADJ': ['GP', 'ADJ', 'ADV'], 'GN': ['GADJ', 'DET', 'NOM', 'GP']}
latex
Afior	est	une grande planète			

groupe departP : [u'Afior']
groupe depart : [u'Afior']
groupe departP : [u'une', u'grande', u'plan\xe8te']
groupe depart : [u'une', u'grande', u'plan\xe8te']
le lapin avec Léina	verront	ces planètes			

groupe departP : [u'le', u'lapin', u'avec', u'L\xe9ina']
groupe depart : [u'le', u'lapin', u'avec', u'L\xe9ina']
faire_gp avec Léina fonction 
avec avec
pas de Cas
groupe prep : [u'avec', u'L\xe9ina']
ref \AVEC avec
groupe departP : [u'L\xe9ina']
groupe depart : [u'L\xe9ina']
groupe departP : [u'ces', u'plan\xe8tes']
groupe depart : [u'ces', u'plan\xe8tes']
les étoiles	brillent				dans le ciel

groupe departP : [u'les', u'\xe9toiles']
groupe depart : [u'les', u'\xe9toiles']
faire_gp dans le ciel fonction 
dans dans
pas de Cas
groupe prep : [u'dans', u'le', u'ciel']
ref \DANS dans
groupe departP : [u'le', 

In [137]:
PFM.lexique.formeLexeme.keys()

[u'\xe9toile',
 u'transforme',
 u'regardent',
 u'capturer',
 u'longera',
 u'brilleront',
 u'aider',
 u'enfants',
 u'voit',
 u'transformer',
 u'le',
 u'noir',
 u'plan\xe8tes',
 u'devenir',
 u'la',
 u'L\xe9ina',
 u'PRO',
 u'maison',
 u'eux',
 u'briller',
 u'regarder',
 u'mangent',
 u'sur',
 u'pelle',
 u'vert',
 u'rambarde',
 u'Anouk',
 u'atterrir',
 u'lancera',
 u'sont',
 u'tartines',
 u'deviendra',
 u'devient',
 u'voir',
 u'obscurit\xe9',
 u'pommes',
 u'capturera',
 u'l',
 u'jouent',
 u'cri',
 u'licorne',
 u'manger',
 u'd\xe9couvrir',
 u'en',
 u'lance',
 u'longeront',
 u'joueront',
 u'aide',
 u'lancent',
 u'jouer',
 u'DEM',
 u'captureront',
 u'brille',
 u'aidera',
 u'est',
 u'cassera',
 u'poussera',
 u'filles',
 u'prennent',
 u'pousser',
 u'casser',
 u'ailes',
 u'verront',
 u'pousse',
 u'casse',
 u'aile',
 u'd\xe9couvrent',
 u'cris',
 u'formule',
 u'atterriront',
 u'\xe9crivent',
 u'casseront',
 u'pousseront',
 u'les',
 u'longent',
 u'chevaliers',
 u'magicien',
 u'magiques',
 u'regarder

##### Fichiers secondaires

In [138]:
if typeKalaba!="Kanonik":
    sortie="images"
    if "RightLeft" in globals() and RightLeft:
        syntagmes=syntagmesRL
    else:
        syntagmes=syntagmesLR
    print syntagmes

    lexemesLocaux=set()
    if ecriture_nom.endswith("csv"):
        try:
            ecriture_file = codecs.open(ecriture_nom,"r","utf8")
        except IOError:
            print 'I could not open the file', ecriture_nom
            sys.exit()      
        else:
            try:
                cloze_file = codecs.open(cloze_nom,"r","utf8")
            except IOError:
                print 'I could not open the cloze file', cloze_nom
                sys.exit()
            else:
                ecriture={}
                for line in cloze_file.readlines():
                    line=line.strip()
                    if not line.startswith("#"):
                        elementsCloze=line.split(";")
                        ecriture[elementsCloze[0]]=elementsCloze[5]
                cloze_file.close()
                exemples=[]
                accumulateur=[]
                vocabulaire=[]
    #            prononciationExtrait=[]
                faire_phrases(ecriture_file,sortie=sortie)
                ecriture_file.close()

    localClozes=filtrerCloze(lexemesLocaux)
    with codecs.open(serie+complementPhrases+"%s-Clozes"%sortie.capitalize()+".txt", 'wb',encoding="utf8") as output:
        for ligne in localClozes:
            output.write(ligne+"\n")

In [139]:
if typeKalaba!="Kanonik":
    sortie="ordre"
    syntagmes=syntagmesLR
    print syntagmes

    lexemesLocaux=set()
    if ordre_nom.endswith("csv"):
        try:
            ordre_file = codecs.open(ordre_nom,"r","utf8")
        except IOError:
            print 'I could not open the file', ordre_nom
            sys.exit()      
        else:
            try:
                cloze_file = codecs.open(cloze_nom,"r","utf8")
            except IOError:
                print 'I could not open the cloze file', cloze_nom
                sys.exit()
            else:
                ordre={}
                for line in cloze_file.readlines():
                    line=line.strip()
                    if not line.startswith("#"):
                        elementsCloze=line.split(";")
                        ordre[elementsCloze[0]]=elementsCloze[5]
                cloze_file.close()
                exemples=[]
                accumulateur=[]
                vocabulaire=[]
    #            prononciationExtrait=[]
                faire_phrases(ordre_file,sortie=sortie)
                ordre_file.close()

    localClozes=filtrerCloze(lexemesLocaux)
    with codecs.open(serie+complementPhrases+"%s-Clozes"%sortie.capitalize()+".txt", 'wb',encoding="utf8") as output:
        for ligne in localClozes:
            output.write(ligne+"\n")

In [140]:
if typeKalaba!="Kanonik":
    sortie="mots"
    if "RightLeft" in globals() and RightLeft:
        syntagmes=syntagmesRL
    else:
        syntagmes=syntagmesLR
    print syntagmes

    lexemesLocaux=set()
    if mots_nom.endswith("csv"):
        try:
            mots_file = codecs.open(mots_nom,"r","utf8")
        except IOError:
            print 'I could not open the file', mots_nom
            sys.exit()      
        else:
            try:
                cloze_file = codecs.open(cloze_nom,"r","utf8")
            except IOError:
                print 'I could not open the cloze file', cloze_nom
                sys.exit()
            else:
                motsIsoles={}
                for line in cloze_file.readlines():
                    line=line.strip()
                    if not line.startswith("#"):
                        elementsCloze=line.split(";")
                        motsIsoles[elementsCloze[0]]=elementsCloze[5]
                cloze_file.close()
                exemples=[]
                accumulateur=[]
                vocabulaire=[]
    #            prononciationExtrait=[]
                faire_phrases(mots_file,sortie=sortie)
                mots_file.close()

    localClozes=filtrerCloze(lexemesLocaux)
    with codecs.open(serie+complementPhrases+"%s-Clozes"%sortie.capitalize()+".txt", 'wb',encoding="utf8") as output:
        for ligne in localClozes:
            output.write(ligne+"\n")

#### Conclusion

In [141]:
time.sleep(1)
if variante=="": 
    print "faire la correction"
    nextVariante="-Corr"
    time.sleep(1)
    ding()
    time.sleep(1)
    ding()
    time.sleep(1)
    ding()
else:
    print u"ok pour cette étape"
    del nextVariante
    ding()

ok pour cette étape


# Post-traitements

In [142]:
def yamlDump(nFile,content):
    with open(nFile, 'w') as output:
        yaml.dump(content, output, default_flow_style=False,allow_unicode=True)

    with open(nFile, 'r') as input:
        yamlLines=input.readlines()

    yamlText="".join(yamlLines)
    yamlText=re.sub(r"!!python/unicode","",yamlText)
    yamlText=re.sub(r"\n\s*-\s*",", ",yamlText)
    yamlText=re.sub(r":,\s*",": ",yamlText)
    yamlText=re.sub(r": *([^\n]+)",": [\g<1>]",yamlText)


    with open(nFile, 'w') as output:
        output.write(yamlText)
    return

In [143]:
for k in cellules:
    cellules[k]=list(cellules[k])
cellules

{'ADJ': [u'noir.Ina.SG',
  u'magique.Ina.SG',
  u'long.Ina.SG',
  u'vert.Ina.SG',
  u'blanc.Ani.SG',
  u'gros.Ina.SG',
  u'sombre.Ina.SG',
  u'grand.Ina.SG',
  u'bleu.Ina.SG'],
 'DET': [u'DEF.Hum.PL',
  u'IND.Ina.SG',
  u'DEF.Ani.DU',
  u'IND.Ani.PL',
  u'DEM.Ani.SG',
  u'DEF.Ina.DU',
  u'DEF.Hum.SG',
  u'IND.Ani.SG',
  u'DEF.Ani.PL',
  u'DEM.Ani.DU',
  u'DEM.Ina.SG',
  u'IND.Ina.PL',
  u'DEM.Ina.PL',
  u'DEF.Ani.SG',
  u'DEF.Ina.SG',
  u'DEF.Ina.PL',
  u'DEF.Hum.DU'],
 'NOM': [u'\xe9toile.Ina.Pl',
  u'ciel.Ina.Sg',
  u'maison.Ina.Sg',
  u'enfant.Hum.Du',
  u'rivi\xe8re.Ina.Sg',
  u'obscurit\xe9.Ina.Sg',
  u'stylo.Ina.Sg',
  u'cr\xe2ne.Ina.Pl',
  u'poisson.Ani.Pl',
  u'chevalier.Hum.Du',
  u'tartine.Ina.Sg',
  u'cri.Ina.Pl',
  u'enfant.Hum.Sg',
  u'vent.Ina.Sg',
  u'formule.Ina.Sg',
  u'soleil.Ina.Sg',
  u'poisson.Ani.Du',
  u'tartine.Ina.Du',
  u'oiseau.Ani.Pl',
  u'grotte.Ina.Sg',
  u'lasagne.Ina.Pl',
  u'lapin.Ani.Sg',
  u'L\xe9ina.Hum.Sg',
  u'caillou.Ina.Sg',
  u'plan\xe8te.Ina.Sg

In [144]:
yamlDump(serie+"FilledCells.yaml",cellules)